In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import chardet
import os
import time

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36"
}




def get_club_names(url):
    """Récupère la liste des clubs (nom + lien) d’un championnat."""
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        print(f"❌ Erreur {response.status_code} pour {url}")
        return []

    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table", class_="items")
    if not table:
        print(f"⚠️ Tableau introuvable pour {url}")
        return []

    clubs = []
    rows = table.find("tbody").find_all("tr")
    for row in rows:
        cols = row.find_all("td")
        if len(cols) >= 3:

            # Récupérer le nom du club
            team = cols[1].get_text(strip=True)

            # Récupérer le lien du club
            a_tag = cols[1].find("a", href=True)
            link = "https://www.transfermarkt.com" + a_tag["href"] if a_tag else ""

            clubs.append((team, link))
            print(team, link)

    return clubs


def save_to_csv(clubs, filename="clubs.csv"):
    """Sauvegarde les clubs dans un fichier CSV."""
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["Championnat", "Club"])
        for championnat, club in clubs:
            writer.writerow([championnat, club])
    print(f"\n💾 {len(clubs)} clubs sauvegardés dans {os.path.abspath(filename)}")


def lire_championnats_csv(fichier_csv):
    """Lit la liste des championnats depuis le CSV d’entrée."""
    with open(fichier_csv, "rb") as f:
        encodage = chardet.detect(f.read())["encoding"]
    with open(fichier_csv, "r", encoding=encodage) as f:
        reader = csv.reader(f)
        next(reader)  # ignorer l’en-tête
        for row in reader:
            if len(row) >= 3 and row[2].startswith("https"):
                yield row[0].strip(), row[2].strip()


def main():
    all_clubs = []
    fichier_csv = "../../championnat/championnat_finale.csv"
    sortie = "equipe.csv"

    print("🚀 Début du scraping des championnats...\n")

    for championnat, lien in lire_championnats_csv(fichier_csv):
        print(f"🔄 Traitement en cours : {championnat}")
        clubs = get_club_names(lien)
        if clubs:
            for club in clubs:
                all_clubs.append((championnat, club))
            print(f"✅ {len(clubs)} clubs trouvés pour {championnat}\n")
        else:
            print(f"⚠️ Aucun club trouvé pour {championnat}\n")
        time.sleep(1.5)  # petite pause pour éviter le blocage par Transfermarkt

    if all_clubs:
        save_to_csv(all_clubs, sortie)
    else:
        print("❌ Aucun club récupéré.")

    print("\n🏁 Fin du programme.")


if __name__ == "__main__":
    main()
